In [ ]:
%matplotlib notebook

from collections import Counter
from itertools import cycle
import json
import os
import pickle
import re
import sys
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

from imgaug import augmenters as iaa
from keras.applications.inception_v3 import InceptionV3
from keras.applications.resnet50 import preprocess_input, ResNet50
from keras.applications.vgg16 import VGG16
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, TensorBoard, EarlyStopping
from keras.layers import (
    Flatten, GlobalAveragePooling2D, GlobalMaxPooling2D, Input, Dense, Dropout)
from keras.models import Model
from keras.optimizers import Nadam
from pycocotools.coco import COCO
from skimage.transform import resize
from sklearn.metrics import precision_recall_curve, average_precision_score
import keras.backend as K
import keras.initializers
import matplotlib.pyplot as plt
import numpy as np
# from herbicide.utils import vis_square

from abyss_deep_learning.keras.classification import (
    ClassificationDataset, caption_map_gen, multihot_gen, augmentation_gen, PRTensorBoard, Inference)
from abyss_deep_learning.keras.utils import (
    batching_gen, lambda_gen, calc_class_weights, count_labels_multi, count_labels_single, gen_dump_data)
import abyss_deep_learning.abyss_dataset as dataset_model

# Setup Data

In [ ]:
############### CONFIGURE ALL VARIABLES IN THIS CELL ########################
# num_classes assumed from caption_map entries
# image_dims = (224, 224, 3) # Preset for Mobilenet
image_dims = (480, 640, 3) # Preset for InceptionV3
batch_size = 2
NN_DTYPE = np.float32

# maps caption strings to class numbers (ensure minimal set of class numbers)
# eg use {0, 1, 2} not {4, 7, 8}

# Caption type can be either "single" or "multi".
# This sets up various parameters in the system.
# If conversion between single and multi is required this should be done explicitly and presented
# in a separate json file. The internal representation of all the labels is one-hot encoding.
caption_type = "multi" 
caption_map = {
    "IP": 0,
#     "JD_ML": 1,
#     "DD": 2,
#     "JD_S": 3,
    "ED": 1
}



class AbyssCaptionTranslator(ClassificationDataset.AnnotationTranslator):
    '''Translates the CloudFactory labels into a form that works with this script'''
    def filter(self, annotation):
        return 'caption' in annotation and 'type' in annotation and annotation['type'] == 'class_labels'
    def translate(self, annotation):
        return annotation['caption'].split(",")
translator = AbyssCaptionTranslator()

dataset_name = "ip_ed_fromCF"
coco_train = ClassificationDataset(
    caption_map, translator,
    "/data/abyss/projectmax/feature-detection/ip-ed-fromCF/train.json")
coco_val = ClassificationDataset(
    caption_map, translator,
    "/data/abyss/projectmax/feature-detection/ip-ed-fromCF/val.json")
coco_test = ClassificationDataset(
    caption_map, translator,
    "/data/abyss/projectmax/feature-detection/ip-ed-fromCF/val.json")

In [ ]:
print("train", set([tuple(coco_train.load_caption(image['id'])) for image in coco_train.imgs.values()]))
print("val", set([tuple(coco_train.load_caption(image['id'])) for image in coco_val.imgs.values()]))

In [ ]:
##### Convert instances to categories ####

# from abyss_deep_learning.coco_classes import CocoDataset
# ds = CocoDataset.from_COCO(coco_val)
# ds.convert_instances_to_captions()
# ds.save("/data/abyss/projectmax/feature-detection/2/validation3.json")
# # coco_val = ClassificationDataset(caption_map, "/data/abyss/projectmax/feature-detection/2/validation3.json")
# # print(len(coco_train.anns))
# # coco_train.anns[60]

In [ ]:
# Find a balanced set
def balanced_set(coco):
    captions = [caption 
            for ann in coco.anns.values() if 'caption' in ann
           for caption in ann['caption'].split(',') if caption != "background"]
    smallest_caption, smallest_caption_value = min(Counter(captions).items(), key=lambda x: x[1])
    
    unique_captions = np.unique(captions)
#     print("unique_captions", unique_captions)
    # Count how many images are in each label
    images_in_caption = {
        caption: [ann['image_id'] for ann in coco.anns.values() if caption in ann['caption'].split(',')]
        for caption in unique_captions}
    
    for images in images_in_caption.values():
        np.random.shuffle(images)
    
    # Count how many captions are in each image
    captions_in_image = {
        image_id: ([
            caption
            for ann in coco.anns.values() if ann['image_id'] == image_id and 'caption' in ann
            for caption in ann['caption'].split(',') if caption != "background"])
        for image_id in coco.imgs}
    print("captions_in_image")
    print([len(captions) for image_id, captions in captions_in_image.items()])
    
#     print("smallest", smallest_caption, smallest_caption_value)
    balanced = []
    out = {caption: [] for caption in unique_captions}
    
    def add_to_counts(image_id):
        # Increment counts for all captions in image
        for caption in captions_in_image[image_id]:
            out[caption].append(image_id)
        # Remove image_id from all images_in_caption
        for images in images_in_caption.values():
            if image_id in images:
                images.pop(images.index(image_id))
    
    while any([len(out[caption]) < smallest_caption_value for caption in unique_captions]):
        least = min(out.items(), key=lambda x: len(x[1]))
        image_id = images_in_caption[least[0]].pop()
        add_to_counts(image_id)
        
    out = [j
           for i in out.values()
          for j in i]
    return set(out)

balanced_image_ids_train = balanced_set(coco_train)
balanced_image_ids_val = balanced_set(coco_val)
balanced_image_ids_test = balanced_set(coco_test)
print("balanced train set size", len(balanced_image_ids_train))
print("balanced val set size", len(balanced_image_ids_val))
print("balanced test set size", len(balanced_image_ids_test))

In [ ]:
caption_map_r = {val: key for key, val in caption_map.items()}
num_classes = len(caption_map)
steps_per_epoch = coco_train.num_images() // batch_size
steps_per_epoch_val = coco_val.num_images() // batch_size
print("Number of classes:", num_classes)
print("Steps per epoch:", steps_per_epoch)
print("Steps per steps_per_epoch_val:", steps_per_epoch_val)

In [ ]:
def preprocess(image, caption):
    image = resize(image, image_dims, preserve_range=True)
    return preprocess_input(image.astype(NN_DTYPE), mode='tf'), caption

def postprocess(image):
    return ((image + 1) * 127.5).astype(np.uint8)
     
def pipeline(gen, aug_config=None):
    return (
        augmentation_gen(
            multihot_gen(
                lambda_gen(
                    caption_map_gen(gen, caption_map, background='background', skip_bg=True)
                , func=preprocess)
            , num_classes=num_classes)
        , aug_config, enable=(aug_config is not None))
    )


aug_config = {
    'flip_lr_percentage': 0.5,
    'flip_ud_percentage': 0.5,
    'affine': {
        "order": 1,
        'scale': {
            "x": (0.8, 1.2),
            "y": (0.8, 1.2)
        },
        "rotate": (-10, 10),
        "shear": (-5, 5),
        "mode": 'constant'
    },
#     'color': {
#         'probability': 1.00,
#         'hue': (0, 0),
#         'saturation': (0, 0),
#         'value': (0, 0)
#     }
}
# aug_config = None # Uncomment to remove augmentation (goes around 50% faster but much worse results)

In [ ]:
train_gen = pipeline(
    coco_train.generator(imgIds=list(balanced_image_ids_train), shuffle_ids=True),
    aug_config=aug_config)
val_gen = pipeline(coco_val.generator(imgIds=list(balanced_image_ids_val), shuffle_ids=True))
test_gen = pipeline(coco_test.generator(imgIds=list(balanced_image_ids_test), shuffle_ids=True))
    
for i, (train, val, test) in enumerate(zip(train_gen, val_gen, test_gen)):
    for data in (train, val, test):
        print(data[0].shape, data[1], (np.min(data[0]), np.max(data[0])))
    plt.figure()
    plt.subplot(1, 3, 1)
    plt.imshow(postprocess(train[0]))
    plt.title(', '.join([caption_map_r[int(cap_id)] for cap_id in np.argwhere(train[1])]))
    
    plt.subplot(1, 3, 2)
    plt.imshow(postprocess(val[0]))
    plt.title(', '.join([caption_map_r[int(cap_id)] for cap_id in np.argwhere(val[1])]))
    
    plt.subplot(1, 3, 3)
    plt.imshow(postprocess(test[0]))
    plt.title(', '.join([caption_map_r[int(cap_id)] for cap_id in np.argwhere(test[1])]))
    
    if i >= 0:
        break
print("Left to right: ground truth samples from train, val test")        

In [ ]:
# This cell intentionally left blank due to display bug above.

In [ ]:
count_function = count_labels_single if caption_type == "single" else count_labels_multi

for label, gen, coco, balanced_image_ids in zip(
        ["train", "val", "test"],
        [train_gen, val_gen, test_gen],
        [coco_train, coco_val, coco_test],
        [balanced_image_ids_train, balanced_image_ids_val, balanced_image_ids_test]):
    data = gen_dump_data(gen, len(balanced_image_ids))
    counter = count_function(data)
    print(label, counter)

val_data = gen_dump_data(val_gen, len(balanced_image_ids_val))
test_data = val_data
class_weights = None
# Uncomment below line to use class weights, not needed if using balanced_set
# class_weights = calc_class_weights(gen_dump_data(train_gen, len(balanced_image_ids_train)), caption_type)

print("training class weights:")
print(class_weights)

print("Binary accuracy if you were to output all 0s")
print((val_data[1] == 0).sum() / val_data[1].size)

# Setup model

In [ ]:
def create_new_head(
    base_model, num_classes, caption_type, model_params, opt_params=None):
    '''make sure base_model has include_top=False. If loss=None then it is determined.'''
    
    if not opt_params:
        opt_params = {"optimizer": Nadam(clipnorm=1)}
    
    if model_params.loss is None:
        if caption_type == "single":
            opt_params['loss'] = "categorical_crossentropy" 
        elif caption_type == "multi":
            # weights = np.array([
                # i[1] for i in sorted(model_params.class_weights.items())])[np.newaxis, ...] \
                # if model_params.class_weights else 1.0
            opt_params['loss'] = 'binary_crossentropy'
    else:
        opt_params['loss'] = model_params.loss
        
    if caption_type == "single":
        activation = "softmax" 
    else:
        activation = "sigmoid"
    
    # add a global spatial average pooling layer
    x = base_model.output
    
    if model_params.pooling == 'avg':
        x = GlobalAveragePooling2D()(x)
    elif model_params.pooling == 'max':
        x = GlobalMaxPooling2D()(x)
    else:
        x = Flatten()(x)
    for _ in range(model_params.num_hidden_layers):
        x = Dense(model_params.num_hidden_neurons, activation=activation,
                  kernel_initializer=keras.initializers.he_uniform())(x)
#         x = relu(x, alpha=0.3, max_value=2.5)
        if model_params.dropout:
            x = Dropout(model_params.dropout)(x)
            
    predictions = Dense(
        num_classes,
        activation=activation,
        kernel_initializer=keras.initializers.he_uniform(),
        name='class_logits')(x)

    for layer in base_model.layers:
        layer.trainable = model_params.train_features
    model = Model(inputs=base_model.input, outputs=predictions)
    
    for layer in model.layers:
        if not layer.trainable:
            continue
        if hasattr(layer, 'kernel_regularizer') and model_params.kernel_regularizer:
            layer.kernel_regularizer = model_params.kernel_regularizer
        if hasattr(layer, 'activity_regularizer') and model_params.activity_regularizer:
            layer.activity_regularizer = model_params.activity_regularizer
    
    print("Compiling model:")
    print("   activation:", activation)
    print("   optimizer:", opt_params)
    model.compile(**opt_params, metrics=['binary_accuracy'])
    return model

In [ ]:
def get_gradients(model):
    """Get the gradients of the loss with respect to the weights."""
    weights = [tensor for tensor in model.trainable_weights 
               if model.trainable_weights]
    return weights, model.optimizer.get_gradients(model.total_loss, weights)

def evaluate_model(model, test_data, thresh=0.5):
    def multi_label_decision(y_true, y_pred):
        return (y_true > thresh) == (y_pred > thresh)
    def single_label_decision(y_true, y_pred):
        return np.argmax(y_true, axis=-1) == np.argmax(y_pred, axis=-1)
    decision_function = single_label_decision if caption_type == 'single' else multi_label_decision

    Y_true = test_data[1]
    Y_pred = model.predict(test_data[0])
    TP = decision_function(Y_true, Y_pred)
    acc = np.count_nonzero(TP) / TP.size
    
    print("Test using {:d} samples:".format(len(test_data[0])))
    print("accuracy", acc)
    return Y_true, Y_pred, TP

def display_performance(Y_true, Y_pred, TP):
    precision = dict()
    recall = dict()
    average_precision = dict()
    for i in range(num_classes):
        precision[i], recall[i], _ = precision_recall_curve(Y_true[:, i],
                                                            Y_pred[:, i])
        average_precision[i] = average_precision_score(Y_true[:, i], Y_pred[:, i])

    # A "micro-average": quantifying score on all classes jointly
    precision["micro"], recall["micro"], _ = precision_recall_curve(Y_true.ravel(),
        Y_pred.ravel())
    average_precision["micro"] = average_precision_score(Y_true, Y_pred,
                                                         average="micro")
    print('Average precision score, micro-averaged over all classes: {0:0.2f}'
          .format(average_precision["micro"]))

    z = np.all((Y_pred > 0.5) == Y_true, axis=1)
    acc = np.count_nonzero(z) / z.size
    print("exact accuracy", acc)
    z = ((Y_pred > 0.5) == Y_true)
    acc = np.count_nonzero(z) / z.size
    print("binary accuracy", acc)
    
    # setup plot details
    colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

    plt.figure(figsize=(8, 10))
    f_scores = np.linspace(0.2, 0.8, num=4)
    lines = []
    labels = []
    for f_score in f_scores:
        x = np.linspace(0.01, 1)
        y = f_score * x / (2 * x - f_score)
        l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
        plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

    lines.append(l)
    labels.append('iso-f1 curves')
    l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
    lines.append(l)
    labels.append('micro-average Precision-recall (area = {0:0.2f})'
                  ''.format(average_precision["micro"]))

    for i, color in zip(range(num_classes), colors):
        l, = plt.plot(recall[i], precision[i], color=color, lw=2)
        lines.append(l)
        labels.append('{0} (area = {1:0.2f})'
                      ''.format(caption_map_r[i], average_precision[i]))

    fig = plt.gcf()
    fig.subplots_adjust(bottom=0.25)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.title('Micro Average Precision vs. Recall')
    plt.legend(lines, labels, loc=(0, -.4), prop=dict(size=14))
    plt.show()
    plt.savefig(model_plot_path, dpi=150)
    
def save_model(model, name, class_map_r, prediction_type,
               model_weights_path, model_def_path, model_info_path, history,
               test_metrics=None, description=""):
    from abyss.utils import JsonNumpyEncoder
    def merged(a, b):
        merged = dict(a)
        merged.update(b)
        return merged
        
    model_info = {
        "name": name,
        "description": description,
        "weights": model_weights_path,
        "prediction_type": caption_type,
        "model": model_def_path,
        "classes": class_map_r,
        "architecture": {
            "backbone": "inceptionv3",
            "logit_activation": model.get_layer("class_logits").activation.__name__,
            "input_shape": image_dims
        },
        "metrics": {
            "loss_function": str(history.model.loss),
            "train": merged(
                history.history,
                {
                    "epoch": history.epoch,
                    "params": history.params
                })
        }
    }
    if test_metrics:
        model_info['metrics']['test'] = test_metrics
    
    print("Writing model def to " + model_def_path)
    with open(model_def_path, "w") as file:
        file.write(model.to_json())
        
    print("Writing model weights to " + model_weights_path)
    model.save_weights(model_weights_path)
    
    print("Writing model info to " + model_info_path)
    with open(model_info_path, "w") as file:
        file.write(json.dumps(model_info, cls=JsonNumpyEncoder))

# Model selection

In [ ]:
def hamming_loss(y_true, y_pred):
    return K.mean(y_true * (1 - y_pred) + (1 - y_true) * y_pred)

def setup_callbacks(log_dir, hist=False):
    !mkdir -p $log_dir/models
    best_path = os.path.join(log_dir, "models/best.{epoch:03d}-{val_loss:.4f}.h5")
    callbacks=[
        ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=6, cooldown=0, verbose=1),
        ModelCheckpoint(
            best_path, monitor='val_loss', verbose=1,
            save_best_only=True, save_weights_only=True, mode='auto', period=1),
            ModelCheckpoint(
                best_path, monitor='val_loss', verbose=1,
                save_best_only=False, save_weights_only=True, mode='auto', period=20),
        PRTensorBoard(
            log_dir=log_dir, 
            histogram_freq=(hist or 0), batch_size=batch_size,
            write_graph=False,
            write_grads=(hist is not None),
            write_images=False),
            EarlyStopping(
                monitor='val_loss', min_delta=0.0, patience=20, verbose=1, mode='auto')
    ]
    return callbacks

def go(epochs, class_weights, initial_epoch=0):
    return model.fit_generator(
        batching_gen(train_gen, batch_size=batch_size),
        validation_data=tuple(val_data),
        steps_per_epoch=steps_per_epoch,
        validation_steps=steps_per_epoch_val,
        class_weight=class_weights,
        callbacks=callbacks, 
        epochs=epochs,
        verbose=1, initial_epoch=initial_epoch, workers=10)

def check_gradients(model):
    grad_test = None
    for image, label in train_gen:
        grad_test = (image, label)
        break
    rates = []
    weights, grads = get_gradients(model)
    feed_dict = {
        "class_logits_sample_weights:0": np.ones(2),
        "input_1:0": grad_test[0][np.newaxis, ...],
        "class_logits_target:0": grad_test[1][np.newaxis, ...]
    }
    for i, (w, g) in enumerate(zip(weights, grads)):
        if 'bias' in w.name:
            continue
        grad_norm = np.linalg.norm(g.eval(feed_dict, K.get_session()))
        weight_norm = np.linalg.norm(w.eval(K.get_session()))
        rate = grad_norm / weight_norm
        rates.append(rate)
    if np.mean(rates) < 5e-4 or np.mean(rates) > 3e-1: # These values change with network structure
        print("Bad gradients ({:.3e}).".format(np.mean(rates)))
        return False
    return True

In [ ]:
# Modify this class to change search parameters
class ExperimentParameters(object):
    def __init__(self, data_name, model_name, batch_size):
        from keras.regularizers import l1_l2
        self.data_name = data_name
        self.learning_rate = 10 ** np.random.uniform(-7, -3)
        self.loss = 'binary_crossentropy'
        self.dropout = 0.5
        self.train_features = False
        self.pooling = 'avg'
        self.num_hidden_layers = 1
        self.num_hidden_neurons = 1024
        self.pretrained_weights = 'imagenet'
        self.class_weights = None
        self.model_name = model_name
        self.batch_size = batch_size
        self.kernel_regularizer = l1_l2(0, 10 ** np.random.uniform(-3, -1))
        self.activity_regularizer = l1_l2(10 ** np.random.uniform(-3, -1), 0)

    def experiment_name(self):
        return str(self)
    
    def __str__(self):
        return '-'.join([
            ':'.join([key, str(value)]) 
            for key, value in [
                ("data", self.data_name),
                ("model", self.model_name),
                ("head", "{:d}x{:d}".format(self.num_hidden_layers, self.num_hidden_neurons)),
                ("train", 'all' if self.train_features else 'heads'),
                ("from", str(self.pretrained_weights)),
                ("loss", self.loss.__name__ if callable(self.loss) else str(self.loss)),
                ("init_lr", "{:.3e}".format(self.learning_rate)),
                ("act_reg", "{:.3e},{:.3e}".format(params.activity_regularizer.l1, params.activity_regularizer.l2)),
                ("ker_reg", "{:.3e},{:.3e}".format(params.kernel_regularizer.l1, params.kernel_regularizer.l2)),
                ("dropout", "{:.1f}".format(self.dropout or 0)),
                ("pool", str(self.pooling)),
                ("CW", str(True if self.class_weights else False))]])

In [ ]:
search_output_dir = "/data/log/cnn/fd/wednesday2" # Change this output dir.
base_model = [ResNet50, InceptionV3, VGG16][0] # Choose one
history_data = {}
for attempt_no in range(100):
    K.clear_session()
    params = ExperimentParameters(dataset_name, base_model.__name__, batch_size)
    base_model = base_model(include_top=False, weights=params.pretrained_weights, input_shape=image_dims)
    model = create_new_head(
        base_model, num_classes, caption_type, params, 
        opt_params={'optimizer': Nadam(clipnorm=1, clipvalue=1)}
    )

    experiment_name = str(params) + "-{:03d}".format(np.random.randint(0, 999))
    log_dir = os.path.join(search_output_dir, experiment_name)
    model_def_path = os.path.join(log_dir, "model_def.json")
    model_weights_path = os.path.join(log_dir, "model_weights.h5")
    model_info_path = os.path.join(log_dir, "model.json")
    model_plot_path = os.path.join(log_dir, "precision-recall.png")
    print(experiment_name)
    print("Training: {:d} layers".format(len([1 for layer in model.layers if layer.trainable])))
    print(log_dir)
    
    callbacks = setup_callbacks(log_dir, hist=None)
    K.set_value(model.optimizer.lr, params.learning_rate)
    history_data[experiment_name] = go(1, params.class_weights)

#     if not check_gradients(model): # Disabled as I only know what to expect for VGG16
#         print("Bad gradients. Continuing to next model.")

    callbacks = setup_callbacks(log_dir, hist=None)
    history_data[experiment_name] = go(29, params.class_weights, initial_epoch=1)
    (Y_true, Y_pred, TP) = evaluate_model(model, test_data, thresh=0.5)
    display_performance(Y_true, Y_pred, TP)
    
    save_model(
        model, name=experiment_name,
        class_map_r=caption_map_r, prediction_type=caption_type,
        model_weights_path=model_weights_path, model_def_path=model_def_path, model_info_path=model_info_path,
        test_metrics=None, history=history_data[experiment_name],
        description="Test model for 5 FDs"
    )
    
with open(os.path.join(search_output_dir, "history-100epoch.pkl"), "wb") as file:
    pickle.dump({key: history.history for key, history in history_data.items()}, file)
### Should you need to load this pkl:
# with open(os.path.join(search_output_dir, "history-100epoch.pkl"), "rb") as file:
#     history = pickle.load(file)

In [ ]:
lrs = [key[1] for key, val  in sorted(history_data.items(), key=lambda x: x[0])]
val_loss = [history.history['val_loss'][-1] for key, history  in sorted(history_data.items(), key=lambda x: x[0])]
loss = [history.history['loss'][-1] for lr, history  in sorted(history_data.items(), key=lambda x: x[0])]
acc = [history.history['binary_accuracy'][-1] for lr, history  in sorted(history_data.items(), key=lambda x: x[0])]
val_acc = [history.history['val_binary_accuracy'][-1] for lr, history  in sorted(history_data.items(), key=lambda x: x[0])]

plt.figure()
plt.subplot(1, 2, 1)
plt.semilogx(lrs, loss, '.b', label='loss')
plt.semilogx(lrs, val_loss, '.r', label='val_loss')
plt.legend()
plt.title("Loss Vs. LR (100 Epoch)")
plt.subplot(1, 2, 2)
plt.semilogx(lrs, acc, '.b', label='binary_accuracy')
plt.semilogx(lrs, val_acc, '.r', label='val_binary_accuracy')
plt.legend()
plt.title("Accuracy Vs. LR (100 Epoch)")
plt.tight_layout()

# Continue training specific model

In [ ]:
model_instance = "data:ip_ed_fromCF-model:resnet50-head:1x1024-train:heads-from:imagenet-loss:binary_crossentropy-init_lr:3.803e-07-act_reg:9.917e-03,0.000e+00-ker_reg:0.000e+00,1.166e-02-dropout:0.5-pool:avg-CW:False-296"
model_best_weight = "best.300-0.7164.h5"
model_initial_learning_rate = 1e-7 ##### IMPORTANT TO SET THIS FROM TENSORBOARD
num_epoch = 200
class_weights = None # Can't currently resume training with imbalance data #TODO

model_weights_in_path = os.path.join(search_output_dir, model_instance, "models", model_best_weight)
log_dir = os.path.join(search_output_dir, model_instance, "continued")
best_path = os.path.join(log_dir, "models/best.{epoch:03d}-{val_loss:.4f}.h5")
model_def_path = os.path.join(log_dir, "model_def.json")
model_info_path = os.path.join(log_dir, "model.json")
model_plot_path = os.path.join(log_dir, "precision-recall.png")
!mkdir -p $log_dir/models

In [ ]:
K.clear_session()
model = Inference(os.path.join(search_output_dir, model_instance, "model.json")).model
model.load_weights(os.path.join(search_output_dir, model_instance, "model_weights.h5"))

model.compile( # TODO, load this from JSON, manually change this if you are doing single label
    Nadam(clipnorm=1),
    loss='binary_crossentropy',
    metrics=['binary_accuracy'])

print("Training: {:d} layers".format(len([1 for layer in model.layers if layer.trainable])))
print(log_dir)

callbacks = setup_callbacks(log_dir, hist=None)
K.set_value(model.optimizer.lr, model_initial_learning_rate)
history_data[experiment_name] = go(num_epoch, class_weights, initial_epoch=0)
(Y_true, Y_pred, TP) = evaluate_model(model, test_data, thresh=0.5)
display_performance(Y_true, Y_pred, TP)

save_model(
    model, name=experiment_name,
    class_map_r=caption_map_r, prediction_type=caption_type,
    model_weights_path=model_weights_path, model_def_path=model_def_path, model_info_path=model_info_path,
    test_metrics=None, history=history_data[experiment_name],
    description="Test model for 5 FDs"
)

# Below is untested with new changes, don't use

# Model Testing

In [ ]:
TP_mask = np.logical_and.reduce(TP, axis=1)
right = test_data[0][TP_mask]
wrong = test_data[0][~TP_mask]
wrong.shape
plt.figure()
vis_square(wrong)
plt.title("Incorrectly Predicted")
plt.figure()
vis_square(right)
plt.title("Correctly Predicted")

# Binary coded the labels then count them wrt TP/FP
coded = np.sum(test_data[1][~TP_mask] * 2 ** np.arange(num_classes)[::-1], axis=1).astype(int)
print("binary coded class error count:", dict(sorted(Counter(coded).items(), key=lambda x: x[0])))
coded = np.sum(test_data[1][TP_mask] * 2 ** np.arange(num_classes)[::-1], axis=1).astype(int)
print("binary coded class correct count:", dict(sorted(Counter(coded).items(), key=lambda x: x[0])))
print(Y_pred[~TP_mask])

# Learning Curve

In [ ]:
def learning_curve(dataset, lr, steps, val_data, log_dir):
    def save_model(path):
        print("Saving", path)
        os.makedirs(os.path.dirname(path), exist_ok=True)
        model.save_weights(path)
    def setup_callbacks():
        return [
                ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, cooldown=5, verbose=1),
                ModelCheckpoint(
                    model_best_path, monitor='val_loss', verbose=1,
                    save_best_only=True, save_weights_only=True, mode='auto', period=1),
#                 ModelCheckpoint(
#                     best_path, monitor='val_loss', verbose=1,
#                     save_best_only=False, save_weights_only=True, mode='auto', period=50),
                PRTensorBoard(
                    log_dir=model_log_dir,
                    histogram_freq=0,
                    batch_size=batch_size,
                    write_graph=False,
                    write_grads=False,
                    write_images=False),
        #         EarlyStopping(
        #             monitor='val_loss', min_delta=0.0, patience=40, verbose=1, mode='auto')
        ]
    def create_new_model(load_base=False):
        K.clear_session()
        model = create_new_head(
            InceptionV3(include_top=False, weights='imagenet', input_shape=image_dims),
            num_classes, caption_type, opt_params={'optimizer': Nadam()},
            class_weights=None, train_features=False, l2_reg=None)
        if load_base:
            print("Loading base model")
            model.load_weights(base_model_path, by_name=True)
        return model

    def train():
        print("Training")
        K.set_value(model.optimizer.lr, lr)
        history[subset_size] = model.fit_generator(
            batching_gen(gen, batch_size=batch_size),
            validation_data=tuple(val_data),
            steps_per_epoch=(subset_size // batch_size),
            validation_steps=steps_per_epoch_val,
            class_weight=model_class_weights,
            callbacks=setup_callbacks(), 
            epochs=50,
            verbose=1)
    model_class_weights = None
    model = None
    model_path = None
    image_ids = [image['id'] for image in dataset.imgs.values()]
    np.random.shuffle(image_ids)
    num_images = len(image_ids)
    print("num_images", num_images)
    history = {}
    base_model_path = os.path.join(log_dir, "base", "weights.h5")
    model_path = base_model_path
    for subset_size in np.linspace(0, num_images, steps + 1).astype(int):
        if subset_size > 0:
            imgIds = image_ids[:subset_size]
            gen = pipeline(
                dataset.generator(shuffle_ids=False, imgIds=imgIds),
                aug_config=None)
            model_class_weights = calc_class_weights(gen, dataset) # TODO

            model_path = os.path.join(log_dir, "subset-of-{:d}/weights.h5".format(subset_size))
            model_log_dir = os.path.dirname(model_path)
            model_best_path = os.path.join(log_dir, "subset-of-{:d}/best.h5".format(subset_size))
            os.makedirs(model_log_dir, exist_ok=True)

            print("learning curve(lr={:.3e}, size={:d})".format(lr, subset_size))
            print("model_log_dir", model_log_dir)
            print("training class weights")
            print(model_class_weights)
        model = create_new_model(load_base=(subset_size > 0))
        if subset_size:
            train()
        save_model(model_path)
    return history

model = None
lr = 1e-5
learning_curve_dir = "/data/log/cnn/fd/learning_curve_5--{:.2e}".format(lr)
lc_history = learning_curve(coco_train, lr, 5, val_data, learning_curve_dir)
val_loss = np.array([(size, h.history['val_loss'][-1]) for size, h in lc_history.items()])
train_loss = np.array([(size, h.history['loss'][-1]) for size, h in lc_history.items()])
plt.figure()
plt.plot(train_loss[:, 0], train_loss[:, 1], 'b.')
plt.plot(val_loss[:, 0], val_loss[:, 1], 'r.')
plt.xlabel("Number of Training Samples")
plt.ylabel("Loss")
plt.savefig(os.path.join(learning_curve_dir, "plot.png"), dpi=150)

In [ ]:
# !rm -R /data/log/cnn/fd/learning-curve/

In [ ]:
images = None
for images, labels in batching_gen(train_gen, batch_size=batch_size):
    print(images.shape, labels.shape)
    
    pred = model.predict(images)
    print(labels)
    print(pred)
    print(K.eval(K.tf.losses.sigmoid_cross_entropy(labels, pred)))
    break

In [ ]:
for unique_label in np.unique(val_data[1], axis=0):
    unique_data = [val_data[0][i] for i in range(len(val_data[0])) if np.all(val_data[1][i] == unique_label)]
    num_data = len(unique_data)
    print(unique_label, num_data)
    plt.figure()
    vis_square(np.array(unique_data))
    plt.title(unique_label)

# Check Update/Weight Ratio

In [ ]:

for i, (w, g) in enumerate(zip(weights, grads)):
    grad_norm = np.linalg.norm(g.eval(feed_dict, K.get_session()))
    weight_norm = np.linalg.norm(w.eval(K.get_session()))
    rate = grad_norm / weight_norm
    print(i, rate)


# Visualize filters

In [ ]:
from herbicide.utils import vis_square
for layer in model.layers:
    if not layer.trainable_weights:
        continue
    for weight in layer.trainable_weights: #  Assumes FD is not trainable
        if 'kernel' not in weight.name:
            continue
        print(weight.name)
        value = K.eval(weight.value())
        print(value.shape)
    
    plt.figure()
    vis_square(value.transpose((3, 0, 1, 2)))
    break


In [ ]:
caption_stats = []
for i, (image, caption) in enumerate(coco_train.generator(imgIds=balanced_image_ids_train)):
    plt.figure()
    plt.imshow(image)
    plt.title(str(caption))
    if i == 10:
        break
    caption_stats.append(caption)

In [ ]:
np.array(caption_stats)